步骤概览：

数据下载与预处理： 使用 AKShare 下载茅台股票从 2000 年 1 月 1 日到今天的日交易数据。
特征构造： 使用交易日前 50 天的收盘价和当天开盘价作为特征，预测当天收盘价的涨跌情况。
数据集划分： 将数据集划分为 70% 的训练集和 30% 的测试集。
模型构建： 建立深度神经网络模型，使用 Softmax 激活函数进行多分类预测。
模型训练与预测： 在训练集上训练模型，并在测试集上进行预测。
模型评估： 计算预测值与真实值之间的偏差，评估模型的准确率、精确率、召回率和 F1 Score 等指标。

In [1]:
import akshare as ak
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, mean_absolute_error

# -------------------------------------
# 1. 数据下载与预处理
# -------------------------------------
start_date = "20000101"
end_date = datetime.today().strftime("%Y%m%d")

# 使用 AKShare 下载茅台股票（股票代码 600519）的日交易数据
df = ak.stock_zh_a_hist(symbol="600519", period="daily", start_date=start_date, end_date=end_date, adjust="")

# 将“日期”列转换为 datetime 类型，并按日期升序排序
df['日期'] = pd.to_datetime(df['日期'])
df.sort_values('日期', inplace=True)
df.reset_index(drop=True, inplace=True)

# -------------------------------------
# 2. 构造样本特征与标签
# -------------------------------------
features = []
labels = []

for i in range(50, len(df)):
    past_50_close = df.loc[i-50:i-1, '收盘'].values.tolist()  # 前50天的收盘价
    today_open = df.loc[i, '开盘']  # 当天开盘价
    sample_features = past_50_close + [today_open]
    features.append(sample_features)
    
    # 标签：1 表示上涨，0 表示下跌
    label = 1 if df.loc[i, '收盘'] > today_open else 0
    labels.append(label)

X = np.array(features)
y = np.array(labels)

# -------------------------------------
# 3. 划分训练集和测试集
# -------------------------------------
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=False)

# -------------------------------------
# 4. 构建深度神经网络模型
# -------------------------------------
class StockPredictor(nn.Module):
    def __init__(self):
        super(StockPredictor, self).__init__()
        self.fc1 = nn.Linear(51, 128)  # 输入层到第一个隐藏层
        self.fc2 = nn.Linear(128, 64)  # 第一个隐藏层到第二个隐藏层
        self.fc3 = nn.Linear(64, 2)    # 第二个隐藏层到输出层
        self.dropout = nn.Dropout(0.5)  # Dropout层，防止过拟合

    def forward(self, x):
        x = torch.relu(self.fc1(x))   # 第一隐藏层 + ReLU激活
        x = self.dropout(x)           # Dropout层
        x = torch.relu(self.fc2(x))   # 第二隐藏层 + ReLU激活
        logits = self.fc3(x)          # 输出层，返回原始logits
        return logits

# 实例化模型
model = StockPredictor()

# -------------------------------------
# 5. 模型训练设置
# -------------------------------------
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
epochs = 50
batch_size = 32

# 将数据转换为 PyTorch 张量
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# -------------------------------------
# 6. 训练模型
# -------------------------------------
for epoch in range(epochs):
    model.train()
    for i in range(0, len(X_train_tensor), batch_size):
        batch_X = X_train_tensor[i:i+batch_size]
        batch_y = y_train_tensor[i:i+batch_size]
        
        # 前向传播
        logits = model(batch_X)
        loss = criterion(logits, batch_y)
        
        # 反向传播和参数更新
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if epoch % 10 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

# -------------------------------------
# 7. 模型预测与评估
# -------------------------------------
model.eval()
with torch.no_grad():
    logits_test = model(X_test_tensor)
    probabilities = torch.softmax(logits_test, dim=1)
    y_pred = torch.argmax(logits_test, dim=1)

# 计算预测值与真实值之间的偏差
pred_error = mean_absolute_error(y_test_tensor.numpy(), y_pred.numpy())

# 计算评价指标
accuracy = accuracy_score(y_test, y_pred.numpy())
precision = precision_score(y_test, y_pred.numpy())
recall = recall_score(y_test, y_pred.numpy())
f1 = f1_score(y_test, y_pred.numpy())

print("\n深度学习模型评估结果：")
print(f"均方误差 (偏差): {pred_error:.4f}")
print(f"准确率: {accuracy:.4f}")
print(f"精确率: {precision:.4f}")
print(f"召回率: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")


ModuleNotFoundError: No module named 'torch'